### 사업보고서 "재무제표 주석" HTML 다운로드한 문서에서 값 읽기
* 파일명: <corp_code>_<recpt_no>_fs_notes.html

In [3]:
# 금액 단위 찾기
def get_unit(text):
    if '(단위:백만원)' in text or '[단위:백만원]' in text: unit = '백만원'
    elif '(단위:천원)' in text or '[단위:천원]' in text: unit = '천원'
    elif '(단위:원)' in text or '[단위:원]' in text: unit = '원'
    else: unit = None   
    return unit

# 콤마와 괄호를 제거한다.
def str2num(x):
    if "," in x: x = x.replace(",","")
    if x.startswith("("): x = x.lstrip("(")
    if x.endswith(")"): x = x.rstrip(")")
    return float(x)

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
path_dir = 'D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_files/'
file_name = '00127802_20150330000271_fs_notes.html'
file_path = path_dir + file_name
with open(file_path, "r") as file:
    html_doc = file.read()

val_a = val_b = 0.0
unit = None

# 금액 단위
soup = BeautifulSoup(html_doc, 'html.parser')
doc_text = soup.text
trimmed_text = doc_text.replace(" ","")     # 문서에 있는 모든 공백 제거
unit = get_unit(trimmed_text)

# 확정급여채무의 현재가치, 사외적립자산의 공정가치
doc_data = pd.read_html(html_doc)
for df in doc_data:
    row = df.shape[0]
    for i in range(0, row):
        item = str(df.iloc[i,0])
        if item.__contains__('확정급여채무의 현재가치'):
            str_a = str(df.iloc[i,1])
            if str_a == "-": val_a = 0.0
            else: val_a += str2num(str_a)       # 값을 합한다.
        elif item.__contains__('사외적립자산의 공정가치'):                      
            str_b = str(df.iloc[i,1])
            if str_b == "-": val_b = 0.0
            else: val_b = str2num(str_b)
        else:
            continue    
    if (val_a != 0.0) and (val_b != 0.0):
        break          
    
print(unit, val_a, val_b, sep=' ')

원 24306779070.0 15839581933.0
